# Module 3 DBSCAN Practice


The activities in this notebook are similar to what you seen in the lab notebook. Most of the activities are either partially complete for you to finish them while some of them might require you to work out the whole code for the activity. 

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler

Generate sample data

**make_blobs(): ** Generates isotropic Gaussian blobs for clustering. 

**centers:**  parameter is used to specify number of centers to generate. It takes input in the form of either "int or array of shape [n_centers, n_features]". 

**cluster_std :** The standard deviation of the clusters.

**random_state :** int, RandomState instance or None, optional (default=None)
If int, random_state is the seed used by the random number generator; If RandomState instance, random_state is the random number generator; If None, the random number generator is the RandomState instance used by np.random.

In [ ]:
# centers of clusters
centers = [[1, 1], [-1, -1], [1, -1]]

# Create blobs with 3 different centers
Data, labels_true = make_blobs(n_samples=750, centers=centers, cluster_std=0.4, random_state=0)
print(Data[1:10])

So the function make_blobs() generated samples 75. samples of data. Data has the x,y coordicates of blobs created and labels_true has the labels to which center the blob belongs to. Below function "StandardScaler().fit_transform(X)" standardizes Data by converting them to zero mean and scaling to unit variance. 

**Activity 1: ** Use the StandardScalar() function to normalize Data.

In [ ]:
# Your answer for activity 1 goes here...

Data = <what goes in here>
Data[1:10]

### Compute DBSCAN

**Activity 2: ** Run the DBSCAN algorithm on Data to construct a DBSCAN object that requires a minimum of 10 data points in a neighborhood of radius 0.3 to be considered a core point.

In [ ]:
# Your answer for activity 2 goes here...

db = DBSCAN(<what goes in here>).fit(<what goes in here>)
db

In [ ]:
print(db.labels_[1:20])
labels_true[1:20]

If you look at the labels returned by dbscan, they are almost same as the labels returned by make_blobs(). There are some -1's apart from {0,1,2}. The -1's represent samples/blobs which couldn't be included in any of the clusters. You can call them outliers in this context but they really are not outliers. They just didnt fit the criteria the algorithm is using to cluster the points. 

Lets generate a vector equal to size of number of labels and filled with boolean values. np.zeros_like() will create a zeros vector but we specified data type as boolean. Since zeros represent False, the vector is populated with False values. 

**Referenec: ** [numpy.zeros_like()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros_like.html)

In [ ]:
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[1:10]

Below displayed is the list of samples indices which were formed into clusteres. Notice that index 5 is missing in the first 10 samples indicating sample 5 was not fitted into any cluster.

In [ ]:
db.core_sample_indices_[1:20]

**Activity 3: ** Assign "True" to the indexes in core_samples_mask list for the samples where dbscan was able to cluster the points. 

In [ ]:
# Your answer for activity 3 goes here...

core_samples_mask[<what goes in here>] = True
core_samples_mask[1:10]

So DBScan was able to label the samples with one of the 4 values {-1, 0, 1, 2}. The values {0, 1, 2 } are labels for valid clusters formed. Below piece of code is reducing the number of clusters by 1 if there are any -1 labels in the list. That will give the actual count of number of clusters formed.

In [ ]:
labels = db.labels_

In [ ]:
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

In [ ]:
print('Estimated number of clusters: %d' % n_clusters_)

In [ ]:
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))

**Reference: ** [metrics.homogeneity_score()](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.homogeneity_score.html)

Homogeneity metric of a cluster labeling given a ground truth. This function will check accuracy of the predictions. The first argument is the true labels which are stored in the list labels_true. labels_true was returned by make_blobs() function while producing the samples. Its the ground truth. The second parameter is the predictions which are stored in the list labels. The function will cross check the labels in both lists to give the accuracy of predictions. 

In [ ]:
print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))

**Reference: ** [sklearn.metrics.completeness_score(labels_true, labels_pred)](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.completeness_score.html)

A clustering result satisfies completeness if all the data points that are members of a given class are elements of the same cluster. Both homogeneity and completeness scores have positive values between 0.0 and 1.0, larger values being desirable.

In [ ]:
print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(Data, labels))

From wiki, The silhouette value is a measure of how similar an object is to its own cluster (cohesion) compared to other clusters (separation). The silhouette values range from -1 to 1, where a high value (i'e closer to 1) indicates that the object is well matched to its own cluster and poorly matched to neighboring clusters. If most objects have a high value, then the clustering configuration is appropriate. If many points have a low or negative value, then the clustering configuration may have too many or too few clusters.

The silhouette can be calculated with any distance metric, such as the Euclidean distance or the Manhattan distance.

**Reference: ** [Python documentation for silhoutte](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html)

In the cell below, first we are assigning unique values {-1, 0, 1, 2} to variable unique_labels. Then using [matplotlib colormap](http://matplotlib.org/api/cm_api.html) module we are generating different colors scheme for each unique label. Spectral is a kind of color map. It is used where the data has a critical middle value. The map consists of two colours with changes in lightness and saturation.

In [ ]:
unique_labels = set(labels)
colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))

**Reference: **
- [Color maps in Matplotlib](http://www.futurile.net/2016/03/31/colormaps-in-matplotlib/)
- [Color maps reference](http://matplotlib.org/examples/color/colormaps_reference.html)
- [Setting the color of a matplotlib plot](http://chrisalbon.com/python/set_the_color_of_a_matplotlib.html)

------

**zip():** This function takes two equal-length collections, and merges them together in pairs. 

**Activity 4: ** Use this on unique_labels and colors below to zip them and assign the result to indexes variable. 

In [ ]:
indexes= zip(<what goes in here>, <what goes in here>)
list(indexes)

The forloop is iterating 4 times in below piece of code. We know samples are labelled as -1 if DBscan could not label them. In each iteration, if the sample label is equal to -1, then it is assigned the color code 'k' which represents black. A new array "class_member_mask" is created to store the indexes with True/False values. If the label value matches k(-1 or 0 or 1 or 2 ) then class_member_mask will have True otherwise False. 

In first iteration, match is an array which has indexes of samples where label is equal to 0 in both class_member_mask and core_samples_mask. nomatch will have indexes of samples where label is equal to 0 in class_member_mask but its not equal to 0 in core_samples_mask. The values in match array are plotted on the map using one a color and values in nomatch are plotted in black color. This repeats for 4 iterations  

In [ ]:
import matplotlib.pyplot as plt

for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = 'k'

    class_member_mask = (labels == k)

    match = Data[class_member_mask & core_samples_mask]
    plt.plot(match[:, 0], match[:, 1], 'o', markerfacecolor=col, markeredgecolor='k', markersize=14)

    nomatch = Data[class_member_mask & ~core_samples_mask]
    plt.plot(nomatch[:, 0], nomatch[:, 1], 'o', markerfacecolor=col, markeredgecolor='k', markersize=6)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()